## Other kinds of MANTO entities and ties

MANTO has records for other entities than people, and many more kinds of ties than just parent-child relationships. We haven't used these features much ourselves, so support is minimal, but it would be exciting to try. 

If you'd like to use MANTO more extensively (or if you're a MANTO developer) please get in touch—we'd love to expand this module.

Here's one example that exploits MANTO's location-based entities and relations:

In [ ]:
from dicesapi import DicesAPI, SpeechGroup
from dicesapi import manto
import pandas as pd

api = DicesAPI(logfile='dices.log')

I happen to know that the MANTO ID for Thebes is `'8253960'`. I found it by searching MANTO's web interface for the place, and then looking at the record data printed at the bottom of the page:

![image](img/manto_footer.png)

I can use the ID to create a new MantoEntity instance. The DICES client treats MANTO IDs as strings, so remember to wrap it in quotation marks.

In [ ]:
manto_place = manto.getMantoID('8253960')
print(manto_place)

Now we can use `getTies()` to find MANTO character entities that have relationships with Thebes. MANTO uses numeric IDs to refer to kinds of relationships as well as to entities. For example, "son of" is `'31764'`.

I haven't found an easy way to search for these by name. Instead, I dig around in the JSON data for some MANTO record (use `manto.dlMantoData(id)` to get the raw record) and compare it with the same record on MANTO's web site to deduce which IDs refer to which relationships. If you want help with a certain relationship, let me know!

### Example: characters who died at Thebes

The relationship 'place of death' turns out to be `'32529'`. That is, if we search for ties to Thebes with this ID, we get back MANTO records for characters who died there.

The `MantoEntity.getTies()` method will return records that have a given tie (or any of a list of ties). By default, it downloads the whole entity record from MANTO and creates a MantoEntity object for each result. Pass it `as_ent=False` to just get the IDs. This is faster.

In [ ]:
manto_ids = manto_place.getTies('32529', as_ent=False)
print(manto_ids)

Now we can use these MANTO IDs for example to filter a CharacterGroup.

In [ ]:
characters = api.getCharacters().filterMantos(manto_ids)

print(len(characters), 'characters who died at', manto_place.name, '\n')

for c in characters:
    print(c.name)

Then use these characters to find speeches:

In [ ]:
speeches = SpeechGroup([s for c in characters for s in api.getSpeeches(spkr_id=c.id)], api=api)

In [ ]:
print(len(speeches), 'speeches by characters who died at', manto_place.name)

pd.DataFrame(dict(
    work = f'{s.author.name} {s.work.title}',
    first_line = s.l_fi,
    last_line = s.l_la,
    speaker = s.getSpkrString(),
    addressee = s.getAddrString(),
) for s in speeches).to_clipboard()

### 🤔 Experiment

Try repeating the steps above using the MANTO ID for a different city. For example, Troy's ID is `'8194710'`.